<a href="https://colab.research.google.com/github/Harinadhavasala/Chatbot_Code/blob/main/Copy_of_Data_lake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
import pandas as pd

In [86]:

df = pd.read_excel("casagrand.xls")

In [ ]:
df=df[["LastName","FirstName","Email"]]

In [87]:
text_cols = df.select_dtypes(include="object").columns
df[text_cols] = df[text_cols].fillna("")

In [113]:
import re
for col in ['FirstName', 'LastName']:
    df[col] = df[col].astype(str)
    df[col] = df[col].str.replace(r'[^a-zA-Z. ]', '', regex=True)



In [116]:
df.head(10)

,LastName,FirstName,Salutation,Name,Company,Phone,MobilePhone,Email,SYS_Mobile__c,WA_Formatted_Phone__c,...,Seventh_Phone__c,name_not_present,only_dots_name,only_initial_name,invalid_name,valid_names,email_not_present,invalid_email_format,duplicate_email,valid_email
0,as chakravarthi,,,As Chakravarthi,As Chakravarthi,919962154351,9.199622e+11,aschakrav@gmail.com,9.199622e+11,9.199622e+11,...,NaN,False,False,False,False,True,False,False,False,True
1,raveendra,meda,,Meda Raveendra,Meda Raveendra,919962041716,9.199620e+11,mrreddy@ramcadds.in,9.199620e+11,9.199620e+11,...,NaN,False,False,False,False,True,False,False,False,True
2,kannan,,Mr,Kannan,Kannan,918189923399,9.181899e+11,kannan.ican1891@gmail.com,9.181899e+11,9.181899e+11,...,NaN,False,False,False,False,True,False,False,False,True
3,ramyaa,,Mr,Ramyaa,Ramyaa,919962746446,9.199627e+11,ramyaa.doc@gmail.com,9.199627e+11,9.199627e+11,...,NaN,False,False,False,False,True,False,False,False,True
4,vasan.k,,Mr,Vasan.K,Vasan.K,918939149522,9.189391e+11,vasankm@yahoo.com,9.189391e+11,9.189391e+11,...,NaN,False,False,False,False,True,False,False,False,True
5,sundar,,Mr,sundar,sundar,919962502711,9.199625e+11,ksundar2001@gmail.com,9.199625e+11,9.199625e+11,...,NaN,False,False,False,False,True,False,False,False,True
6,pandiyan,,Mr,Pandiyan,Pandiyan,919790947384,9.197909e+11,acp3012@gmail.com,9.197909e+11,9.197909e+11,...,NaN,False,False,False,False,True,False,False,False,True
7,k.uma,,Mr,K.Uma,K.Uma,919840891528,9.198409e+11,kuma189@yahoo.com,9.198409e+11,9.198409e+11,...,NaN,False,False,False,False,True,False,False,False,True
8,vinodh kumar,,,Vinodh Kumar,Vinodh Kumar,919940194192,9.199402e+11,vinoths99401@gmail.com,9.199402e+11,9.199402e+11,...,NaN,False,False,False,False,True,False,False,False,True
9,soundar,,,Soundar,Soundar,919677023477,9.196770e+11,phyraj2008@gmail.com,9.196770e+11,9.196770e+11,...,NaN,False,False,False,False,True,False,False,False,True


In [117]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("NameValidation") \
    .getOrCreate()


In [118]:
spark_df = spark.createDataFrame(df)


In [125]:
from pyspark.sql import functions as F

spark_df = spark_df.withColumn(
    "full_name",
    F.trim(F.concat_ws(" ", F.col("FirstName"), F.col("LastName")))
)


In [126]:
spark_df.show(truncate=False)



+---------------+---------------+----------+---------------------------+---------------------------+------------+----------------+-------------------------------+----------------+---------------------+-------------------------------+--------------------+----------------+----------------+---------------+----------+----------+----------+----------+----------+----------+-----------+---------------+--------------+---------------+--------------+---------------+--------------+----------------+----------------+--------------+-----------------+------------+-----------+-----------------+--------------------+---------------+-----------+---------------------------+
|LastName       |FirstName      |Salutation|Name                       |Company                    |Phone       |MobilePhone     |Email                          |SYS_Mobile__c   |WA_Formatted_Phone__c|Email__c                       |Encrypted_SMobile__c|Mobile_Phone__c |Phone__c        |Mobile3__c     |Mobile4__c|Mobile5__c|Mobile6__c|

In [127]:
import pyspark.sql.functions as F

spark_df = (
    spark_df

    # ------------------------------------------------------------------
    # 0. Name not present (highest priority)
    # ------------------------------------------------------------------
    .withColumn(
        "name_not_present",
        F.when(
            F.col("full_name").isNull() |
            (F.trim(F.col("full_name")) == ""),
            True
        ).otherwise(False)
    )
    .withColumn(
        "only_dots_name",
        F.when(
            (F.col("name_not_present") == False) &
            (F.col("full_name").rlike(r"^\s*\.{1,}\s*$")),
            True
        ).otherwise(False)
    )

    .withColumn(
        "only_initial_name",
        F.when(
            (F.col("name_not_present") == False) &
            (F.col("full_name").rlike(r"^\s*\.?[A-Za-z]\.?\s*$")),
            True
        ).otherwise(False)
    )

    # ------------------------------------------------------------------
    # 3. Invalid / blocking names (ONLY if name is present)
    # ------------------------------------------------------------------
    .withColumn(
        "invalid_name",
        F.when(
            (F.col("name_not_present") == False) &
            (F.col("full_name").rlike(
                r"(?i)\b(do not call|dont call|no call|remove|xyz|abcd|abc|test)\b"
            )),
            True
        ).otherwise(False)
    )

    # ------------------------------------------------------------------
    # 4. Final valid flag
    # ------------------------------------------------------------------
    .withColumn(
        "valid_names",
        F.when(
            (F.col("name_not_present") == False) &
            (F.col("only_dots_name") == False) &
            (F.col("only_initial_name") == False) &
            (F.col("invalid_name") == False),
            True
        ).otherwise(False)
    )
)


In [123]:
spark_df = (
    spark_df
    .withColumn(
        "FirstName",
        F.when(F.col("invalid_name") == True, F.lit(""))
         .otherwise(F.col("FirstName"))
    )
    .withColumn(
        "LastName",
        F.when(F.col("invalid_name") == True, F.lit(""))
         .otherwise(F.col("LastName"))
    )
)


In [124]:
spark_df = (
    spark_df
    .withColumn(
        "FirstName",
        F.when(F.col("only_dots_name") == True, F.lit(""))
         .otherwise(F.col("FirstName"))
    )
    .withColumn(
        "LastName",
        F.when(F.col("only_dots_name") == True, F.lit(""))
         .otherwise(F.col("LastName"))
    )
)


In [128]:
from pyspark.sql import functions as F

spark_df = (
    spark_df
    # convert FirstName and LastName to lowercase
    .withColumn("FirstName", F.lower(F.col("FirstName")))
    .withColumn("LastName", F.lower(F.col("LastName")))

    # drop full_name column
    .drop("full_name")
)


In [129]:
sparkdf = spark_df.drop("only_dots_name")

In [137]:
spark_df.show(10,truncate=False)


+---------------+---------+----------+---------------+---------------+------------+----------------+-------------------------+----------------+---------------------+-------------------------+--------------------+----------------+----------------+---------------+----------+----------+----------+----------+----------+----------+-----------+---------------+--------------+---------------+--------------+---------------+--------------+----------------+----------------+--------------+-----------------+------------+-----------+-----------------+--------------------+---------------+-----------+
|LastName       |FirstName|Salutation|Name           |Company        |Phone       |MobilePhone     |Email                    |SYS_Mobile__c   |WA_Formatted_Phone__c|Email__c                 |Encrypted_SMobile__c|Mobile_Phone__c |Phone__c        |Mobile3__c     |Mobile4__c|Mobile5__c|Mobile6__c|Mobile7__c|Mobile8__c|Mobile9__c|Mobile10__c|Country_Code__c|First_Phone__c|Second_Phone__c|Third_Phone__c|Fourt

In [131]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

EMAIL_REGEX = (
    r'^(?=.{1,254}$)(?=.{1,64}@)'
    r'[A-Za-z0-9_%+-]+(\.[A-Za-z0-9_%+-]+)*'
    r'@'
    r'[A-Za-z0-9-]+(\.[A-Za-z0-9-]+)*\.[A-Za-z]{2,}$'
)

# Preserve original order
spark_df = spark_df.withColumn("_row_order", F.monotonically_increasing_id())

# --------------------------------------------------
# 1. Email not present
# --------------------------------------------------
spark_df = spark_df.withColumn(
    "email_not_present",
    F.when(
        F.col("Email").isNull() |
        (F.trim(F.col("Email")) == "") |
        (F.lower(F.trim(F.col("Email"))) == "nan"),
        True
    ).otherwise(False)
)

# --------------------------------------------------
# 2. Invalid email format (regex only)
# --------------------------------------------------
spark_df = spark_df.withColumn(
    "invalid_email_format",
    F.when(
        (F.col("email_not_present") == False) &
        (~F.trim(F.col("Email")).rlike(EMAIL_REGEX)),
        True
    ).otherwise(False)
)


email_window = Window.partitionBy(F.lower(F.trim(F.col("Email"))))

spark_df = spark_df.withColumn(
    "duplicate_email",
    F.when(
        (F.col("email_not_present") == False) &
        (F.col("invalid_email_format") == False) &
        (F.count("*").over(email_window) > 1),
        True
    ).otherwise(False)
)

# --------------------------------------------------
# 4. Final valid email flag
# --------------------------------------------------
spark_df = spark_df.withColumn(
    "valid_email",
    F.when(
        (F.col("email_not_present") == True) |
        (F.col("invalid_email_format") == True) |
        (F.col("duplicate_email") == True),
        False
    ).otherwise(True)
)

# --------------------------------------------------
# Restore original order
# --------------------------------------------------
spark_df = spark_df.orderBy("_row_order").drop("_row_order")


In [132]:
spark_df = spark_df.withColumn(
    "Email",
    F.when(F.col("invalid_email_format") == True, F.lit(""))
     .otherwise(F.col("Email"))
)


In [133]:
spark_df.show(10,truncate=False)


+---------------+---------+----------+---------------+---------------+------------+----------------+-------------------------+----------------+---------------------+-------------------------+--------------------+----------------+----------------+---------------+----------+----------+----------+----------+----------+----------+-----------+---------------+--------------+---------------+--------------+---------------+--------------+----------------+----------------+--------------+-----------------+------------+-----------+-----------------+--------------------+---------------+-----------+
|LastName       |FirstName|Salutation|Name           |Company        |Phone       |MobilePhone     |Email                    |SYS_Mobile__c   |WA_Formatted_Phone__c|Email__c                 |Encrypted_SMobile__c|Mobile_Phone__c |Phone__c        |Mobile3__c     |Mobile4__c|Mobile5__c|Mobile6__c|Mobile7__c|Mobile8__c|Mobile9__c|Mobile10__c|Country_Code__c|First_Phone__c|Second_Phone__c|Third_Phone__c|Fourt

In [134]:
df = spark_df.toPandas()


In [136]:
df.to_csv("output3.csv", index=False)
